# API Request notebook for climate and air quality data from Copernicus Atmosphere Monitoring Service (CAMS) and Copernicus Climate Change Service (C3S)

This notebook is used to download data derived from meteorological and air quality models. It's divided into 2 parts:
- The first part requests data using **CDS API** Python library. It's used to download data concerning air quality variables from Copernicus Atmosphere Monitoring Service (CAMS) model.
- The second part requests data using **Google Earth Engine** Python API library. It's used to dowload data concerning meteorological variables obtained from the Compernicus Climate Change Servince ERA5-land hourly Reanalysis model.

### Link to Copernicus Services:
- Link to [ADS Atmosphere Copernicus (CAMS)](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)
- Link to [CDS Climate Copernicus (C3S)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)

### How to access the CDS API:
It's possible to change **key** and **value** provided from the website in the **.cdsapirc** file as explained in the following link: https://ads.atmosphere.copernicus.eu/api-how-to 

Otherwise **key** and **value** can be used as input directly inside the **cdsapi** API request.

### Reference material:<br> 
- GitHub Repository containing examples from ECMWF: https://github.com/ecmwf-projects/copernicus-training
- Example: https://github.com/ecmwf-projects/copernicus-training/blob/master/2021-02-Copernicus-ECMWF-data-tutorial.ipynb

### Import libraries and API authentication

In [ ]:
import cdsapi
import os
import geopandas as gpd
import xarray as xr
import json
import ipywidgets as widgets
import ee
import geemap
import datetime

import warnings
warnings.filterwarnings('ignore')

# Current working directory path:
cwd = os.getcwd()

# Import functions defined for DDUST project:
from functions import my_methods

In [ ]:
# Google Earth Engine authentication (a Google account is required):
ee.Authenticate()

In [ ]:
# Initialize Google Earth Engine API:
ee.Initialize()

In [ ]:
# CDSAPI: Importing key/value from .json file for API authentication:
f = open('keys.json')
keys = json.load(f)
c_atm = cdsapi.Client(keys["ATM_ID"], keys["ATM_KEY"])

### Set starting and end date for data request:

The date selection is done through the **date.json** file that contains as **keys** the crop name (e.g. cereal_week) and as **values** a list of the corresponding manuring/observation weeks. The  **date.json** file is the same for all the project's notebooks:

In [ ]:
d = open('date.json') #Open .json file where manuring weeks and year are saved
date = json.load(d)
year = date['year']
custom_week = date['custom_week']

The calendar **manuring_periods** functions allows to create and visualize the calendar with corresponding manuring weeks:

In [ ]:
calendar = my_methods.manuring_periods(year, custom_week)

In [ ]:
# Select start and end date of the corresponding selected week:
start_date = str(datetime.datetime.strptime((str(year)+'-'+custom_week[0]), "%Y-%m-%d").date())
end_date = str(datetime.datetime.strptime((str(year)+'-'+custom_week[1]), "%Y-%m-%d").date())
print("The starting date is", start_date,"and the ending date is" , end_date,". The date is define as yyyy-mm-dd.")

### Import Bounding Box

In [ ]:
area_path = cwd + '/grid/grid_0_1.gpkg'
area = gpd.read_file(area_path).to_crs(4326)
bounds = area.total_bounds
roi = ee.Geometry.BBox(bounds[0],bounds[1],bounds[2],bounds[3])

In [ ]:
era5_scale = 1000

- - -

# CDAPI Data Requests for Copernicus Atmosphere Monitoring Service (CAMS)

This part contains the requests for the following variables:
- PM2.5
- Ammonia NH3
- Sulphur Dioxide SO2
- Nitrogen Dioxide NO2
- Nitrogen Monoxide NO
- Carbon Monoxide CO
- Ozone O3
- Dust
- PM10
- NMVOCs (Non-Methane Volatile Organic Compounds)

Each section just requires hourly data in the time range (start & end date) for each variable. The data are required in the bounding box only, and a mean value it's calculated. The output format is a netCDF file.

# [PM2.5 - Particulate Matter](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [ ]:
pm25_path = cwd+r'\temp\pm25_cams.nc'
pm25 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'particulate_matter_2.5um',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    pm25_path
    )

In [ ]:
pm25 = xr.open_dataset(pm25_path)

In [ ]:
pm25_mean = pm25.mean(dim='time')

In [ ]:
pm25_mean.to_netcdf(cwd+r'\temp\pm25_cams.nc')

- - -

# [NH3 - Ammonia](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [ ]:
nh3_path = cwd+r'\temp\nh3_cams.nc'
nh3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ammonia',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    nh3_path
    )

In [ ]:
nh3 = xr.open_dataset(nh3_path)

In [ ]:
nh3_mean = nh3.mean(dim='time')

In [ ]:
nh3_mean.to_netcdf(cwd+r'\temp\nh3_cams.nc', 'w')

- - -

# [SO2 - Sulphur Dioxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [ ]:
so2_path = cwd + r'/temp/so2_cams.nc'
so2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'sulphur_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    so2_path
    )

In [ ]:
so2 = xr.open_dataset(so2_path)

In [ ]:
so2_mean = so2.mean(dim='time')

In [ ]:
so2_mean.to_netcdf(so2_path)

- - -

# [NO2 - Nitrogen Dioxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [ ]:
no2_path = cwd + r'/temp/no2_cams.nc'
no2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no2_path
    )

In [ ]:
no2 = xr.open_dataset(no2_path)

In [ ]:
no2_mean = no2.mean(dim='time')

In [ ]:
no2_mean.to_netcdf(no2_path)

- - -

# [NO - Nitrogen Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [ ]:
no_path = cwd + r'/temp/no_cams.nc'
no = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no_path
    )

In [ ]:
no = xr.open_dataset(no_path)

In [ ]:
no_mean = no.mean(dim='time')

In [ ]:
no_mean.to_netcdf(no_path)

- - -

# [CO - Carbon Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [ ]:
co_path = cwd + r'/temp/co_cams.nc'
co = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'carbon_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    co_path
    )

In [ ]:
co = xr.open_dataset(co_path)

In [ ]:
co_mean = co.mean(dim='time')

In [ ]:
co_mean.to_netcdf(co_path)

---

# [O3 - Ozone](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [ ]:
o3_path = cwd + r'/temp/o3_cams.nc'
o3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ozone',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    o3_path
    )

In [ ]:
o3 = xr.open_dataset(o3_path)

In [ ]:
o3_mean = o3.mean(dim='time')

In [ ]:
o3_mean.to_netcdf(o3_path)

---

# [Dust](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [ ]:
dust_path = cwd + r'/temp/dust_cams.nc'
dust = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'dust',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    dust_path
    )

In [ ]:
dust = xr.open_dataset(dust_path)

In [ ]:
dust_mean = dust.mean(dim='time')

In [ ]:
dust_mean.to_netcdf(dust_path)

# [PM10 - Particulate Matter](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [ ]:
pm10_path = cwd + r'/temp/pm10_cams.nc'
pm10 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'particulate_matter_10um',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    pm10_path)

In [ ]:
pm10 = xr.open_dataset(pm10_path)

In [ ]:
pm10_mean = pm10.mean(dim='time')

In [ ]:
pm10_mean.to_netcdf(pm10_path)

# [NMVOCs - Non Methane VOCs](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [ ]:
nmvocs_path = cwd + r'/temp/nmvocs_cams.nc'
nmvocs = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'non_methane_vocs',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    nmvocs_path)

In [ ]:
nmvocs = xr.open_dataset(nmvocs_path)

In [ ]:
nmvocs_mean = nmvocs.mean(dim='time')

In [ ]:
nmvocs_mean.to_netcdf(nmvocs_path)

---

# [ECMWF C3S ERA-5 Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY) 
WCMWF C3S ERA-5 Model data are retrieved using Google Earth Engine API.
The data selected are:
 - Air temperature at 2 m above the land surface
 - Precipitation defined as accumulated liquid and frozen water, including rain and snow
 - Pressure defined as weight of all the air in a column vertically above the area of the Earth's surface represented at a fixed point.
 - Northward and eastward wind (u_wind, v_component 10 m above the land surface
 - Soil humidity defined as volume of water in soil layer 1 (0 - 7 cm)

In [ ]:
Map_era5 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_era5

In [ ]:
# Daily mean 2m air temperature
temp_2m = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                   .select('temperature_2m')\
                   .filter(ee.Filter.date(start_date, end_date));

# Daily total precipitation sums
prec = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                  .select('total_precipitation')\
                  .filter(ee.Filter.date(start_date, end_date));

# Daily mean surface pressure
press = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                  .select('surface_pressure')\
                  .filter(ee.Filter.date(start_date, end_date));

# Daily mean 10m u-component of wind
u_wind = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                          .select('u_component_of_wind_10m')\
                          .filter(ee.Filter.date(start_date, end_date));

# Daily mean 10m v-component of wind
v_wind = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                          .select('v_component_of_wind_10m')\
                          .filter(ee.Filter.date(start_date, end_date));


# Daily volumetric soil water content (level 1)
hum = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                          .select('volumetric_soil_water_layer_1')\
                          .filter(ee.Filter.date(start_date, end_date));


# Visualization palette for temperature at 2m
temp_2m_viz  = {
  'min': 220,
  'max': 304,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
};

# Visualization palette for pressure (surface pressure)
press_viz = {
  'min': 65000,
  'max': 120000,
  'palette': [
    '#01FFFF', '#058BFF', '#0600FF', '#DF00FF', '#FF00FF', '#FF8C00', '#FF8C00'
  ]
};

# Visualization palette for total precipitation
prec_viz  = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
};

# Visualization palette for u-component of 10m wind
u_wind_viz = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
}
                           
# Visualization palette for v-component of 10m wind
v_wind_viz = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
};

# Visualization palette for soil humitidy
hum_viz = {
  'min': 0,
  'max': 1,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
};

temp_2m = temp_2m.mean()
press = press.mean()
prec = prec.mean()
u_wind = u_wind.mean()
v_wind = v_wind.mean()
hum = hum.mean()

In [ ]:
#Clip over the ROI previously defined
temp_2m_clip = temp_2m.clip(roi)
press_clip = press.clip(roi)
prec_clip = prec.clip(roi)
u_wind_clip = u_wind.clip(roi)
v_wind_clip = v_wind.clip(roi)
hum_clip = hum.clip(roi)

Map_era5.addLayer(temp_2m_clip, temp_2m_viz, '2m temperature');
Map_era5.addLayer(press_clip, press_viz, 'press');
Map_era5.addLayer(prec_clip, prec_viz, 'precipitation');
Map_era5.addLayer(u_wind_clip, u_wind_viz, 'u wind');
Map_era5.addLayer(v_wind_clip, v_wind_viz, 'v wind');
Map_era5.addLayer(hum_clip, hum_viz, 'soil humitidy');

In [ ]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'temp_2m.tif')
geemap.ee_export_image(temp_2m_clip, filename=filename, scale=era5_scale)

In [ ]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'press.tif')
geemap.ee_export_image(press_clip, filename=filename, scale=era5_scale)

In [ ]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'prec.tif')
geemap.ee_export_image(prec_clip, filename=filename, scale=era5_scale)

In [ ]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'e_wind.tif')
geemap.ee_export_image(u_wind_clip, filename=filename, scale=era5_scale)

In [ ]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'n_wind.tif')
geemap.ee_export_image(v_wind_clip, filename=filename, scale=era5_scale)

In [ ]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'soil_hum.tif')
geemap.ee_export_image(hum_clip, filename=filename, scale=era5_scale)